In [1]:
from sympy import *
init_printing(order='none')
from IPython.display import Markdown

def display_expr(expr, label="", uneval_exprs=[]):
    wild = Wild("wild")
    wild2 = Wild("wild2")
    for e in uneval_exprs:
        expr = expr.subs(e, UnevaluatedExpr(e))
        expr = expr.replace(UnevaluatedExpr(wild)**wild2, UnevaluatedExpr(UnevaluatedExpr(wild)**wild2))
        expr = expr.replace(wild*UnevaluatedExpr(wild2), UnevaluatedExpr(wild)*UnevaluatedExpr(wild2))
    return display(Markdown(f'<div style="text-align: center">\n\n#### {label}\n\n</div>\n\n' + '\\begin{equation}\n' + f'{latex(expr)}' + ' \\end{equation}'))

# Feasibility of a Vacuum Airship

A [vacuum airship](https://en.wikipedia.org/wiki/Vacuum_airship) is a structure that is less dense than air (at [STP](https://en.wikipedia.org/wiki/Standard_temperature_and_pressure)) but does not contain gases that are (or would be) less dense than air at STP.

## The specific strain energy constraint

The energy needed to create the vacuum must be stored in the mechanical deformation/strain energy of the structure.

The energy needed to create a vacuum of volume $V$ within a gas at pressure $\sigma_{air}$ is given by:

In [2]:
sigma_air = symbols("\\sigma_{air}", real=True, finite=True, positive=True)
V = symbols("V", real=True, finite=True, positive=True)

U_air = sigma_air * V

display_expr(U_air, "Energy to create vacuum")

<div style="text-align: center">

#### Energy to create vacuum

</div>

\begin{equation}
V \sigma_{air} \end{equation}


The [strain energy](https://en.wikipedia.org/wiki/Strain_energy) of a structure made of a volume of $V_m$ of material with elastic modulus $E_m$, that is under stress $\sigma_m$ is:

In [3]:
sigma_m = symbols("\\sigma_{m}", real=True, finite=True, positive=True)
V_m = symbols("V_{m}", real=True, finite=True, positive=True)
E_m = symbols("E_{m}", real=True, finite=True, positive=True)
U_strain = sigma_m**2*V_m/(2*E_m)
display_expr(U_strain, "Strain energy")

<div style="text-align: center">

#### Strain energy

</div>

\begin{equation}
\frac{V_{m} \sigma_{m}^{2}}{2 E_{m}} \end{equation}

So:

In [4]:
energy_constraint = Ge(U_strain, U_air)
display_expr(energy_constraint, "Energy constraint")

<div style="text-align: center">

#### Energy constraint

</div>

\begin{equation}
\frac{V_{m} \sigma_{m}^{2}}{2 E_{m}} \geq V \sigma_{air} \end{equation}

The mass $m_{ship}$ of the airship is given by:

In [5]:
m_ship = symbols("m_{ship}", real=True, finite=True, positive=True)
rho_m = symbols("\\rho_{m}", real=True, finite=True, positive=True)
eq_m_ship = Eq(m_ship, V_m*rho_m)
display_expr(eq_m_ship)

<div style="text-align: center">

#### 

</div>

\begin{equation}
m_{ship} = V_{m} \rho_{m} \end{equation}

Solving for $V_m$ gives:

In [6]:
V_m_expr = solve([eq_m_ship], [V_m], dict=True)[0][V_m]
display_expr(Eq(V_m, V_m_expr))

<div style="text-align: center">

#### 

</div>

\begin{equation}
V_{m} = \frac{m_{ship}}{\rho_{m}} \end{equation}

Similarly, the mass $m_{air}$ of the air displaced by the vacuum is given by:

In [7]:
m_air = symbols("m_{air}", real=True, finite=True, positive=True)
rho_air = symbols("\\rho_{air}", real=True, finite=True, positive=True)
eq_m_air = Eq(m_air, V*rho_air)
display_expr(eq_m_air)

<div style="text-align: center">

#### 

</div>

\begin{equation}
m_{air} = V \rho_{air} \end{equation}

Solving for V gives:


In [8]:
V_expr = solve([eq_m_air], [V], dict=True)[0][V]
display_expr(Eq(V, V_expr))

<div style="text-align: center">

#### 

</div>

\begin{equation}
V = \frac{m_{air}}{\rho_{air}} \end{equation}

Substituting the formulas for $V$ and $V_{m}$ into the original inequality gives:


In [9]:
energy_constraint = energy_constraint.subs(V, V_expr).subs(V_m, V_m_expr)
display_expr(energy_constraint)

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{m_{ship} \sigma_{m}^{2}}{2 E_{m} \rho_{m}} \geq \frac{\sigma_{air} m_{air}}{\rho_{air}} \end{equation}


Dividing both sides by $m_{air}$ gives:


In [10]:
energy_constraint2 = Gt(energy_constraint.lhs/m_air, energy_constraint.rhs/m_air)
display_expr(energy_constraint2, "", [sigma_m**2/(2*E_m*rho_m), m_ship/m_air])

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{m_{ship}}{m_{air}} \frac{\sigma_{m}^{2}}{2 E_{m} \rho_{m}} > \frac{\sigma_{air}}{\rho_{air}} \end{equation}


Since $m_{ship}/m_{air} \lt 1$ in order to achieve bouyancy:


In [11]:
specific_energy_constraint = energy_constraint2.subs(m_ship/m_air, 1)
display_expr(specific_energy_constraint)

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{m}^{2}}{2 E_{m} \rho_{m}} > \frac{\sigma_{air}}{\rho_{air}} \end{equation}


The left hand side is the specific strain energy in the material (i.e. the strain energy per unit of mass). It must be greater than the pressure of air divided by the density of air. At STP, $\sigma_{air}=101,325\ kPa$ and $\rho_{air}=1.225\ kg/m^2$, so:


In [27]:
specific_energy_constraint_stp = specific_energy_constraint.subs(sigma_air, 101325).subs(rho_air, 1.225)
specific_energy_constraint_stp = specific_energy_constraint_stp.func(specific_energy_constraint_stp.lhs, N(specific_energy_constraint_stp.rhs, 4))
display_expr(specific_energy_constraint_stp, "Specific Strain Energy Constraint")

<div style="text-align: center">

#### Specific Strain Energy Constraint

</div>

\begin{equation}
\frac{\sigma_{m}^{2}}{2 E_{m} \rho_{m}} > 8.271 \cdot 10^{4} \end{equation}

## Constraints imposed if using an octet truss

The constraint on the specific strain energy of the material applies no matter what structure the material is formed into. In this section, we consider forming the material into an octet truss structure. If each unit of the octet truss is less dense than air is strong enough to resist air pressure, a "solid" structure made from a sufficiently large number of the units and covered with an airtight skin would float. The idea is that the mass of the airtight skin only increases with the surface area of the structure, while the overall mass increases with it's volume. As a result the relative contribution of the mass of the skin is neglible for a sufficiently large structure.


If the struts of the octet-truss are hollow tubes with a wall thickness of $t$, density can be lowered for an equal drop in strength and Young's modulus. For $t \ll d$:

In [13]:
d = symbols("d", real=True, finite=True, positive=True)
t = symbols("t", real=True, finite=True, positive=True)
rho_strut = symbols("\\rho_{strut}", real=True, finite=True, positive=True)
eq_rho_strut = Eq(rho_strut, 2*t/d*rho_m)
display_expr(eq_rho_strut, "", [2*t/d])
sigma_strut = symbols("\\sigma_{strut}", real=True, finite=True, positive=True)
eq_sigma_strut = Eq(sigma_strut, 2*t/d*sigma_m)
display_expr(eq_sigma_strut, "", [2*t/d])
E_strut = symbols("E_{strut}", real=True, finite=True, positive=True)
eq_E_strut = Eq(E_strut, 2*t/d*E_m)
display_expr(eq_E_strut, "", [2*t/d])

<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{strut} = \rho_{m} \frac{2 t}{d} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
\sigma_{strut} = \sigma_{m} \frac{2 t}{d} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
E_{strut} = E_{m} \frac{2 t}{d} \end{equation}


Formulas which relate the density, strength, and elastic modulus of an octet truss to the density, strength, and elastic modulus of the strut material as well as the slenderness ratio of the struts can be found in section 2.1.1 of [a 2020 paper by Lijun Xiao, et al](https://www.researchgate.net/publication/344302750_A_Multi-Cell_Hybrid_Approach_to_Elevate_the_Energy_Absorption_of_Micro-Lattice_Materials). Specifically, if $d$ and $l$ are the diameter and length of a strut respectively, $d \ll l$, but $d/l$ is not so small that the struts buckle before yielding:

In [14]:
rho_oct = symbols("\\rho_{oct}", real=True, finite=True, positive=True)
l = symbols("l", real=True, finite=True, positive=True)
eq_rho_oct_strut = Eq(rho_oct/rho_strut, 3*sqrt(2)*pi/2*(d/l)**2)
display_expr(eq_rho_oct_strut, "", [d/l])

sigma_oct = symbols("\\sigma_{oct}", real=True, finite=True, positive=True)
eq_sigma_oct_strut = Eq(sigma_oct/sigma_strut, rho_oct/(3*rho_strut))
display_expr(eq_sigma_oct_strut, "", [rho_oct/rho_strut])

E_oct = symbols("E_{oct}", real=True, finite=True, positive=True)
eq_E_oct_strut = Eq(E_oct/E_strut, rho_oct/(9*rho_strut))
display_expr(eq_E_oct_strut, "", [rho_oct/rho_strut])


<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\rho_{oct}}{\rho_{strut}} = \frac{3 \pi \sqrt{2}}{2} \left(\frac{d}{l}\right)^{2} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{oct}}{\sigma_{strut}} = \frac{1}{3} \frac{\rho_{oct}}{\rho_{strut}} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{E_{oct}}{E_{strut}} = \frac{1}{9} \frac{\rho_{oct}}{\rho_{strut}} \end{equation}

Combining those formulas with the formulas for the density, strength, and Young's modulus of the strut gives:

In [15]:
eq_rho_oct = Eq(rho_oct, solve([eq_rho_oct_strut, eq_rho_strut], [rho_oct, rho_strut], dict=True)[0][rho_oct])
eq_rho_oct = use(eq_rho_oct, lambda side: side/rho_m, level=1)
display_expr(eq_rho_oct, "", [d*t/l**2])

eq_sigma_oct = eq_sigma_oct_strut.subs(eq_sigma_strut.lhs, eq_sigma_strut.rhs).subs(eq_rho_strut.lhs, eq_rho_strut.rhs)
eq_sigma_oct = use(eq_sigma_oct, lambda side: side*2*t/d, level=1)
display_expr(eq_sigma_oct, "", [rho_oct/rho_m])

eq_E_oct = eq_E_oct_strut.subs(eq_E_strut.lhs, eq_E_strut.rhs).subs(eq_rho_strut.lhs, eq_rho_strut.rhs)
eq_E_oct = use(eq_E_oct, lambda side: side*2*t/d, level=1)
display_expr(eq_E_oct, "", [rho_oct/rho_m])


<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\rho_{oct}}{\rho_{m}} = 3 \pi \sqrt{2} \frac{d t}{l^{2}} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{oct}}{\sigma_{m}} = \frac{1}{3} \frac{\rho_{oct}}{\rho_{m}} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{E_{oct}}{E_{m}} = \frac{1}{9} \frac{\rho_{oct}}{\rho_{m}} \end{equation}

Note that the octet-truss is perfectly efficient -- the specific energy of something made with the octet-truss as a material has the same specific energy as the same thing made from the underlying material. That fact is independent of $d$, $l$, and $t$. This can be seen by solving the last two equations for $\sigma_{oct}$ and $E_{oct}$, seeing that $\rho_{oct} = {\rho_{oct} \over \rho_m} \rho_m$ and using the resulting formulas for $\sigma_{oct}$, $E_{oct}$, and $\rho_{oct}$ in the formula for specific energy of the octet-truss. After cancelling, the result is the same as the formula for the specific energy of the material.

Note that the yield strength of the octet-truss $\sigma_{oct}$ must be at least the pressure exerted by the $\sigma_{air}$, and the density of the octet-truss $\rho_{oct}$ must be less than the density of air $\rho_{air}$:

In [16]:
display_expr(Ge(sigma_oct, sigma_air))
display_expr(Lt(rho_oct, rho_air))


<div style="text-align: center">

#### 

</div>

\begin{equation}
\sigma_{oct} \geq \sigma_{air} \end{equation}

<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{oct} < \rho_{air} \end{equation}

We can combine these two inequalities to get:

In [17]:
specific_strength_oct_constraint = Ge(sigma_oct/rho_oct, sigma_air/rho_air)
display_expr(specific_strength_oct_constraint)

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{oct}}{\rho_{oct}} \geq \frac{\sigma_{air}}{\rho_{air}} \end{equation}

Combining that with the equation for the relative strength of the octet truss gives:

In [18]:
# Solve eq_sigma_oct for sigma_oct and substitute that into specific_strength_oct_constraint
specific_strength_constraint = specific_strength_oct_constraint.subs(sigma_oct, solve([eq_sigma_oct], [sigma_oct], dict=True)[0][sigma_oct])
# Multiply both sides by 3
specific_strength_constraint = use(specific_strength_constraint, lambda s: 3*s, level=1)
display_expr(specific_strength_constraint, "", [sigma_air/rho_air])

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{m}}{\rho_{m}} \geq 3 \frac{\sigma_{air}}{\rho_{air}} \end{equation}

Substituting the values for air at standard temperature and pressure, gives:

In [19]:
specific_strength_constraint_stp = specific_strength_constraint.subs(sigma_air, 101000).subs(rho_air, 1.225)
display_expr(N(specific_strength_constraint_stp, 4), "Specific Strength Constraint")

<div style="text-align: center">

#### Specific Strength Constraint

</div>

\begin{equation}
\frac{\sigma_{m}}{\rho_{m}} \geq 2.473 \cdot 10^{5} \end{equation}

This is still perfectly structurally efficient, and just effectively changes the material used in the octet-truss to one with correspondingly lower density, strength, and Young's modulus. As a result, the constraint on the specific strength of the material is unchanged.

Note however, that there are limits to how small $2t/d$ or $d/l$ can be. Various references indicate that to avoid thin-wall effects $2t/d > 1/10$, and to ensure that the strut doesn't collapse due to buckling before the material yields:

In [20]:
yield_before_buckling_constraint = Lt(sigma_m, pi**2*E_m/(2*l/d)**2)
display_expr(yield_before_buckling_constraint, "", [d/l])

<div style="text-align: center">

#### 

</div>

\begin{equation}
\sigma_{m} < \frac{E_{m} \pi^{2}}{4} \left(\frac{d}{l}\right)^{2} \end{equation}

Rearranging that results in a slenderness constraint:

In [21]:

slenderness_constraint = use(yield_before_buckling_constraint, lambda side: side/(E_m*pi**2/4), level=1)
display_expr(slenderness_constraint, "", [d/l])


<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{4 \sigma_{m}}{\pi^{2} E_{m}} < \left(\frac{d}{l}\right)^{2} \end{equation}

Combining that constraint and the constraint to avoid thin-walls gives:


In [22]:
geometry_constraint = Lt(slenderness_constraint.lhs/10, slenderness_constraint.rhs*2*t/d)
geometry_constraint = use(geometry_constraint, lambda side: side/2, level=1)
display_expr(geometry_constraint)

<div style="text-align: center">

#### 

</div>

\begin{equation}
\frac{\sigma_{m}}{5 \pi^{2} E_{m}} < \frac{d t}{l^{2}} \end{equation}

Solving the relative density formula for the octet-truss for the density of the truss gives:

In [23]:
eq_rho_oct_solved = Eq(rho_oct, solve([eq_rho_oct], [rho_oct], dict=True)[0][rho_oct])
display_expr(eq_rho_oct_solved)

<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{oct} = \frac{3 \pi \rho_{m} d t \sqrt{2}}{l^{2}} \end{equation}

Combining with the geometry constraint and the fact that $\rho_{oct} < \rho_{air}$ buoyancy, gives:

In [24]:
density_constraint = Gt(rho_air, eq_rho_oct_solved.rhs.subs(geometry_constraint.rhs, geometry_constraint.lhs))
display_expr(density_constraint)

<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{air} > \frac{3 \rho_{m} \sigma_{m} \sqrt{2}}{5 \pi E_{m}} \end{equation}

Rearraning so that all of the material properties are on one side gives a constraint on the the product of the material's density $\rho_m$ and the material's yield strain (which is the $\sigma_m/E_m$):

In [25]:
density_strain_constraint = use(density_constraint, lambda side: side/(3*sqrt(2)/(5*pi)), level=1).reversed
display_expr(density_strain_constraint, "", [sigma_m/E_m, rho_air])

<div style="text-align: center">

#### 

</div>

\begin{equation}
\rho_{m} \frac{\sigma_{m}}{E_{m}} < \frac{5 \pi \sqrt{2}}{6} \rho_{air} \end{equation}

Using the density of air at STP:

In [26]:
density_strain_constraint_stp = density_strain_constraint.subs(rho_air, 1.225)
display_expr(N(density_strain_constraint_stp, 4), "Density Strain Constraint", [sigma_m/E_m])


<div style="text-align: center">

#### Density Strain Constraint

</div>

\begin{equation}
\rho_{m} \frac{\sigma_{m}}{E_{m}} < 4.535 \end{equation}